In [ ]:
!pip install sentence_transformers

#Additional Installations
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 761 kB/s 
     |████████████████████████████████| 5.5 MB 33.1 MB/s 
     |████████████████████████████████| 1.3 MB 47.3 MB/s 
     |████████████████████████████████| 182 kB 54.0 MB/s 
     |████████████████████████████████| 7.6 MB 31.6 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=f6c2e13c62306f00236ae6adc400ecf0865d6e556a082503bfc40099bea0e5cf
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9674 sha256=5d3ffd0eab6bf280bfce3a28b49bf5818561b7c0d22625ce02d78b1f5fbee790
  Stored in directory: /root/.cache/pip/wh

## Importing necessary libraries. 
In the final version all imports should be stricly enlisted here.

In [ ]:
import pandas as pd
import numpy as np
import spacy
from scipy import stats
from sklearn import linear_model
from sklearn.feature_extraction.text import TfidfVectorizer

from sentence_transformers import SentenceTransformer, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample
from sentence_transformers import losses
import tensorflow as tf
import torch 
from torch import nn
from torch.utils.data import DataLoader

import csv
import tarfile
import wget
import re
import unicodedata
import nltk
from nltk.corpus import stopwords
from sklearn.svm import SVR

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
stopword=set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


**Preprocess the data**

In [ ]:
def clean_text(text):
  text = re.sub('\S*@\S*\s?', '', text)
  text = re.sub('\s+', ' ', text) 
  text = re.sub("\'", '', text)
  text = re.sub('[^a-zA-Z]', ' ', text)
  text = re.sub('\s+[a-zA-Z]\s+^I', ' ', text)
  text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
  text = re.sub(r'\s+', ' ', text)
  text = re.sub(r'^\s*|\s\s*', ' ', text).strip()
  text = text.lower()
  return text

In [ ]:
df_train=pd.read_csv("/content/F2_Claim_Check_Worthiness_train.csv")

In [ ]:
df_test=pd.read_csv("/content/F2_Claim_Check_Worthiness_test.csv")

In [ ]:
df_train['tweet'] = df_train['tweet'].apply(lambda x : clean_text(x)).values.tolist()
df_test['tweet'] = df_test['tweet'].apply(lambda x : clean_text(x)).values.tolist()

In [ ]:
NON_CONEXTUAL_MODEL_TYPE =TfidfVectorizer(stop_words=stopword,max_features=10) #tf-idf
CONEXTUAL_MODEL_TYPE = SentenceTransformer('paraphrase-MiniLM-L6-v2')  #SBERT
HUGGING_FACE_SENTENCE_TRANSFORMER_MODEL = models.Transformer('sentence-transformers/stsb-mpnet-base-v2')
INPUT_PATH = "/content/Data/stsbenchmark/"
BATCH_SIZE = 16
OUT_DIM_DENSE = 512
NUM_EPOCHS = 2 ## THIS IS FIXED DO NOT CHANGE


Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.69k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [ ]:
from sklearn import svm

In [ ]:
def get_feature_model2(data_frame):

  sentences1=list(data_frame.tweet)
  embeddings1 = model2.encode(sentences1)
  return embeddings1

model2 =CONEXTUAL_MODEL_TYPE

feature_1_train= get_feature_model2(df_train)
X_train=np.array(feature_1_train)

s_test=get_feature_model2(df_test)
X_test=np.array(s_test)



Y_train = np.array(df_train["label"])


regr =svm.SVC()
regr.fit(X_train, Y_train)
pred_test=regr.predict(X_test)


In [ ]:
data = {'label': df_test["id"],
        'id': pred_test}

df_out = pd.DataFrame(data)
df_out.set_index('label', inplace=True)
df_out.to_csv('context_svm.csv')